# Part1

## Q1

- **The Principle**  
  Both functions use the argument-halving identity

  $$
  \ln x \;=\; 2^{n}\,\big(x^{\,1/2^{n}} - 1\big).
  $$

  It follows from $ x^{\,1/2^{n}} = e^{\ln(x)/2^{n}}$ and the Taylor series
  $ e^{u} = 1 + u + \tfrac{u^{2}}{2} + \cdots $ with $ u = \ln(x)/2^{n} $.  
  As $ n $ increases, $ x^{\,1/2^{n}} $to $ 1 $ and the truncation error shrinks like $ O(2^{-n}) $.

---

- **When \( 0 < x < 1 \)**  
  Here $ \ln x < 0 \ $. Repeated square roots still push $  x^{\,1/2^{n}} $ up toward 1 from below, so
  $ x^{\,1/2^{n}} - 1 < 0 $ and

  $$
  2^{n}\big(x^{\,1/2^{n}} - 1\big)
  $$

  correctly produces a negative $ \ln x \ $. (Undefined for $ x \le 0 $).

---

- **Role of n**  
  n counts how many halving steps you take. Larger n makes $ x^{\,1/2^{n}} $ closer to 1, reducing
  the analytic error roughly by a factor $ \approx \tfrac{1}{2} $ per step.  
  Numerically, too large n can amplify rounding errors (see below).

---

- **`naivelog` path**  
  1) Square-root repeatedly: $  y_0 = x  $, $ y_{k+1} = \sqrt{y_k} \Rightarrow y_n = x^{\,1/2^{n}} $.  
  2) Form $ y_n - 1 $ (this is catastrophic cancellation when $ y_n \approx 1 $).  
  3) Multiply by $ 2^{n} $.

---

- **`mylog` path (cancellation-free)**  
  Start with $ z_0 = x - 1 $ and use the identity

  $$
  \sqrt{1+z} - 1 \;=\; \frac{z}{\,1 + \sqrt{1+z}\,}
  $$

  to avoid subtracting nearly equal numbers. Iterate

  $$
  z_{k+1} \;=\; \frac{z_k}{\,1 + \sqrt{1 + z_k}\,}.
  $$

  If $ y_k = \sqrt{1 + z_k} $, then $ z_{k+1} = y_k - 1 $ and $ y_{k+1} = \sqrt{y_k} $.  
  Hence

  $$
  z_n \;=\; x^{\,1/2^{n}} - 1,
  $$

  i.e., the same mathematical quantity as in naivelog, but computed stably at every step.  
  Finally multiply by $ 2^{n} $.


## Q2

In [51]:
import math
import pandas as pd



# Same logic as code provided but with a variable n (to test n = 15,20,25,30,35)
def naivelog_n(x, n):
    y = x
    for _ in range(n):
        y = math.sqrt(y)
    l = y - 1
    for _ in range(n):
        l *= 2
    return l

def mylog_n(x, n):
    z = x - 1
    for _ in range(n):
        z = z / (1 + math.sqrt(1 + z))
    l = z
    for _ in range(n):
        l *= 2
    return l

# Build the result DataFrame for Q2
rows = []
for n in [15, 20, 25, 30, 35]:
    for x in range(1, 15): 
        true_val = math.log(x)
        naive_val = naivelog_n(x, n)
        mylog_val = mylog_n(x, n)
        rows.append({
            "n": n,
            "x": x,
            "math.log(x)": true_val,
            "naivelog": naive_val,
            "mylog": mylog_val,
            "err_naive": abs(naive_val - true_val),
            "err_mylog": abs(mylog_val - true_val),
        })

df = pd.DataFrame(rows)
print(df.to_string(index=False))


 n  x  math.log(x)  naivelog    mylog    err_naive    err_mylog
15  1     0.000000  0.000000 0.000000 0.000000e+00 0.000000e+00
15  2     0.693147  0.693155 0.693155 7.331180e-06 7.331183e-06
15  3     1.098612  1.098631 1.098631 1.841679e-05 1.841679e-05
15  4     1.386294  1.386324 1.386324 2.932494e-05 2.932494e-05
15  5     1.609438  1.609477 1.609477 3.952534e-05 3.952534e-05
15  6     1.791759  1.791808 1.791808 4.898774e-05 4.898774e-05
15  7     1.945910  1.945968 1.945968 5.777956e-05 5.777956e-05
15  8     2.079442  2.079508 2.079508 6.598158e-05 6.598158e-05
15  9     2.197225  2.197298 2.197298 7.366796e-05 7.366796e-05
15 10     2.302585  2.302666 2.302666 8.090244e-05 8.090244e-05
15 11     2.397895  2.397983 2.397983 8.773868e-05 8.773868e-05
15 12     2.484907  2.485001 2.485001 9.422176e-05 9.422176e-05
15 13     2.564949  2.565050 2.565050 1.003897e-04 1.003897e-04
15 14     2.639057  2.639164 2.639164 1.062746e-04 1.062746e-04
20  1     0.000000  0.000000 0.000000 0.

For larger (n), `naivelog` degrades while `mylog` keeps improving.

At n = 30 the worst error of `naivelog` is $ (2.19\times10^{-7}) $ (e.g., (x=3)), but `mylog` is $ (3.24\times10^{-9}) $ which is about **68× smaller**.
* At n = 35 `naivelog`’s worst error blows up to $ (7.09\times10^{-6}) $ (at (x=11)), while `mylog` stays at $ (1.01\times10^{-10}) $ which is **70,000× smaller!!!**.

**Why this happens:**

* In `naivelog`, after n square roots $ y=x^{1/2^n} $ is extremely close to 1. The step `l = y - 1` causes catastrophic cancellation by subtracting two nearly equal numbers, wiping out many significant digits. The final scaling `l *= 2` (n times) then amplifies that rounding noise by (2^{n}). Past some n, rounding dominates and the error grows 
* In `mylog`, each “halve” uses the identity
 $$ 
  \sqrt{1+z}-1 = \frac{z}{1+\sqrt{1+z}},
  $$
  which avoids the subtraction of two nearly equal numbers. Algebraically, after n steps you still have $ z_n = x^{1/2^n}-1 $, but it’s computed stably; only then do you scale by $ 2^{n} $. Hence the method keeps tracking the $ O(2^{-n}) $ improvement instead of being swamped by cancellation.

**The table shows the trade-off :**

1.  Equal performance at (n=15,20,25) (truncation dominates).
2.  Massive advantage for `mylog` at (n=30,35) (round-off dominates `naivelog`): ~**68×** smaller worst error at (n=30); ~**7×10^4×** smaller at (n=35).



## Q3

In [52]:
import math
import pandas as pd

# --- Q3 functions using "argument halving"  ---

# Exponential:  exp(x) ≈ (1 + x/2^n)^(2^n)
def myexp_n(x, n):
    y = 1.0 + x / (2 ** n)
    for _ in range(n):
        y = y * y
    return y

# Sine via half-angle start + repeated doubling:
# start at t = x/2^n with small-angle Taylor, then apply:
#   sin(2θ) = 2 sinθ cosθ,   cos(2θ) = cos^2θ − sin^2θ   (n times)
def mysin_n(x, n):
    t = x / (2 ** n)
    s = t - (t**3)/6 + (t**5)/120
    c = 1 - (t**2)/2 + (t**4)/24 - (t**6)/720
    for _ in range(n):
        s, c = 2*s*c, c*c - s*s
    return s

def mycos_n(x, n):
    t = x / (2 ** n)
    s = t - (t**3)/6 + (t**5)/120
    c = 1 - (t**2)/2 + (t**4)/24 - (t**6)/720
    for _ in range(n):
        s, c = 2*s*c, c*c - s*s
    return c

# --- Showcase with examples ---

n = 20  # number of halvings 

# Exponential examples
xs_exp = [-2.0, -1.0, 0.0, 1.0, 2.0]
rows_exp = []
for x in xs_exp:
    approx = myexp_n(x, n)
    truth = math.exp(x)
    rows_exp.append({"n": n, "x": x, "math.exp(x)": truth, "myexp": approx, "abs_err": abs(approx - truth)})
df_exp = pd.DataFrame(rows_exp)
print(df_exp.to_string(index=False))

# Trig examples on 0..1.6 (≈ just over π/2 )
xs_trig = [k/10 for k in range(0, 17)]  # 0, 0.1, ..., 1.6
rows_sin, rows_cos = [], []
for x in xs_trig:
    s_approx = mysin_n(x, n)
    c_approx = mycos_n(x, n)
    s_truth = math.sin(x)
    c_truth = math.cos(x)
    rows_sin.append({"n": n, "x": x, "math.sin(x)": s_truth, "mysin": s_approx, "abs_err": abs(s_approx - s_truth)})
    rows_cos.append({"n": n, "x": x, "math.cos(x)": c_truth, "mycos": c_approx, "abs_err": abs(c_approx - c_truth)})

df_sin = pd.DataFrame(rows_sin)
df_cos = pd.DataFrame(rows_cos)
print(df_sin.to_string(index=False))
print(df_cos.to_string(index=False))


 n    x  math.exp(x)    myexp      abs_err
20 -2.0     0.135335 0.135335 2.581307e-07
20 -1.0     0.367879 0.367879 1.754151e-07
20  0.0     1.000000 1.000000 0.000000e+00
20  1.0     2.718282 2.718281 1.296183e-06
20  2.0     7.389056 7.389042 1.409342e-05
 n   x  math.sin(x)    mysin      abs_err
20 0.0     0.000000 0.000000 0.000000e+00
20 0.1     0.099833 0.099833 2.467304e-12
20 0.2     0.198669 0.198669 9.819895e-12
20 0.3     0.295520 0.295520 5.233980e-12
20 0.4     0.389418 0.389418 6.837975e-12
20 0.5     0.479426 0.479426 5.551115e-16
20 0.6     0.564642 0.564642 2.000111e-11
20 0.7     0.644218 0.644218 1.593958e-11
20 0.8     0.717356 0.717356 2.519274e-11
20 0.9     0.783327 0.783327 3.589895e-11
20 1.0     0.841471 0.841471 1.776357e-15
20 1.1     0.891207 0.891207 4.411804e-11
20 1.2     0.932039 0.932039 4.246137e-11
20 1.3     0.963558 0.963558 4.375988e-11
20 1.4     0.985450 0.985450 4.876533e-11
20 1.5     0.997495 0.997495 4.369838e-13
20 1.6     0.999574 0.999574

- **Exponential (`myexp_n`)**  
  Uses the identity  

  $$
  e^x = \lim_{n \to \infty} (1 + x / 2^n)^{2^n}.
  $$  

  For finite $n$, we compute $(1 + x / 2^n)^{2^n}$ by squaring $n$ times.  
  As $n$ increases, the truncation error decreases geometrically $O(2^{-n})$, so accuracy improves rapidly.

---

- **Sine and Cosine (`mysin_n`, `mycos_n`)**  
  Begin with a very small angle $t = x / 2^n$.  
  Use the **small-angle Taylor series**:

  $$
  \sin t \approx t - \frac{t^3}{6} + \frac{t^5}{120}, \quad
  \cos t \approx 1 - \frac{t^2}{2} + \frac{t^4}{24} - \frac{t^6}{720}.
  $$

  Then repeatedly apply the **doubling formulas** $n$ times:

  $$
  \sin(2\theta) = 2 \sin\theta \cos\theta, \quad
  \cos(2\theta) = \cos^2\theta - \sin^2\theta.
  $$

  This reconstructs $\sin x$ and $\cos x$ from accurate small-angle approximations.

---

For trigonometric functions, using $x = 0, 0.1, 0.2, \dots, 1.6$ (≈ up to $\pi/2$) avoids large-angle propagation errors.  Within this range, the combination of halving and doubling gives excellent precision without overflow or instability.




## Q4

In [53]:
import math
import pandas as pd

# --- Q4: arctangent via “argument halving” ---

# arctan(x) with n halvings:
#   θ = arctan(x)
#   tan(θ/2) = x / (1 + sqrt(1 + x^2))   (apply n times)
#   arctan(x) = 2^n * arctan( t_n ),     where t_{k+1} = t_k / (1 + sqrt(1 + t_k^2))
# For small t, use a short alternating Taylor series: arctan t ≈ t - t^3/3 + t^5/5 - t^7/7 + t^9/9
def myatan_n(x, n):
    t = float(x)
    for _ in range(n):
        t = t / (1.0 + math.sqrt(1.0 + t*t))  # stable halving step
    # small-angle polynomial (odd terms only)
    t2 = t*t
    s = t - t*t2/3.0 + t*t2*t2/5.0 - t*t2*t2*t2/7.0 + t*t2*t2*t2*t2/9.0
    return s * (2 ** n)

# --- examples ---

n = 20  # number of halvings

# Example grid for x (argument of arctan); modest values keep errors tiny
xs = [k/10 for k in range(0, 17)]  # 0, 0.1, ..., 1.6
rows = []
for x in xs:
    approx = myatan_n(x, n)
    truth = math.atan(x)
    rows.append({"n": n, "x": x, "math.atan(x)": truth, "myatan": approx, "abs_err": abs(approx - truth)})
df = pd.DataFrame(rows)
print(df.to_string(index=False))

# compute π using π = 4 * arctan(1)
rows_pi = []
for n_try in [10, 15, 20, 25, 30]:
    pi_approx = 4.0 * myatan_n(1.0, n_try)
    rows_pi.append({"n": n_try, "pi_approx": pi_approx, "abs_err": abs(pi_approx - math.pi)})
df_pi = pd.DataFrame(rows_pi)
print(df_pi.to_string(index=False))


 n   x  math.atan(x)   myatan      abs_err
20 0.0      0.000000 0.000000 0.000000e+00
20 0.1      0.099669 0.099669 1.387779e-17
20 0.2      0.197396 0.197396 8.326673e-17
20 0.3      0.291457 0.291457 1.110223e-16
20 0.4      0.380506 0.380506 0.000000e+00
20 0.5      0.463648 0.463648 3.330669e-16
20 0.6      0.540420 0.540420 1.110223e-16
20 0.7      0.610726 0.610726 1.110223e-16
20 0.8      0.674741 0.674741 1.110223e-16
20 0.9      0.732815 0.732815 1.110223e-16
20 1.0      0.785398 0.785398 4.440892e-16
20 1.1      0.832981 0.832981 2.220446e-16
20 1.2      0.876058 0.876058 0.000000e+00
20 1.3      0.915101 0.915101 0.000000e+00
20 1.4      0.950547 0.950547 0.000000e+00
20 1.5      0.982794 0.982794 3.330669e-16
20 1.6      1.012197 1.012197 2.220446e-16
 n  pi_approx      abs_err
10   3.141593 1.332268e-15
15   3.141593 8.881784e-16
20   3.141593 1.776357e-15
25   3.141593 1.332268e-15
30   3.141593 1.776357e-15



**Halving identity for arctangent.** From the tangent addition law， doubling/halving formula
  $$
  \arctan x = 2\ \arctan\left(\frac{x}{1+\sqrt{1+x^2}}\right)
  $$
  which reduces the argument’s magnitude each time it’s applied. Repeating it (n) times gives
  $ \arctan x = 2^n,\arctan(t_n) $ with $ t_{k+1}=t_k/(1+\sqrt{1+t_k^2}) $

After enough halvings, $ (t_n) $ is tiny and we evaluate $ \arctan(t_n) $ with a short alternating series
  $$
  \arctan t = t - \frac{t^3}{3} + \frac{t^5}{5} - \frac{t^7}{7} + \cdots
  $$
  which is the standard arctangent power series (rapid for $ |t|\ll 1) $. 

The halving step uses $ t/(1+\sqrt{1+t^2}) $, which avoids subtracting nearly equal numbers (a classic source of catastrophic cancellation). This mirrors the general technique of reformulating expressions to remove ill-conditioned subtractions. 

---

**Computing $ \pi $.** With `myatan_n`, the simplest route is
  $$
  \pi \approx 4\ \arctan(1)
  $$
  so the code prints a small table of $ \pi $ estimates for (n=10,15,20,25,30). (Historically, this is the Gregory–Leibniz specialization of the arctangent series; it converges very slowly, while Machin-like combinations converge much faster.)  

The halving identity above is the arctangent “argument reduction” used to accelerate series evaluation (“Ptolemy–Briggs–style” reductions).


# Part 2

## Q1

In [54]:
import math
import pandas as pd

# --- Part 1 arctan (from earlier), used here to build the lookup table ---
def myatan_n(x, n):
    t = float(x)
    for _ in range(n):
        t = t / (1.0 + math.sqrt(1.0 + t*t))  # argument halving (stable)
    t2 = t*t
    s = t - t*t2/3.0 + t*t2*t2/5.0 - t*t2*t2*t2/7.0 + t*t2*t2*t2*t2/9.0  # small-angle series
    return s * (2 ** n)

# --- Build lookup table atan(2^-k) ---
N_ATAN = 35         # accuracy for arctan used in the table
TABLE_LEN = 40      # number of entries
ATAN_TABLE = [myatan_n(2.0 ** -k, N_ATAN) for k in range(TABLE_LEN)]

# --- CORDIC-style tangent with argument reduction to [0, π/2] ---
PI = math.pi
HALF_PI = PI / 2

def cordic_tan_n(x, n):
    # reduce to [0, π)
    t = math.fmod(x, PI)
    if t < 0:
        t += PI
    # reflect to [0, π/2] and keep the sign via tan(π - t) = -tan(t)
    sign = 1.0
    if t > HALF_PI:
        t = PI - t
        sign = -1.0

    # rotation-mode CORDIC; tan θ ≈ y/x (scale cancels in the ratio)
    xk, yk, zk = 1.0, 0.0, t
    for k in range(n):
        d = 1.0 if zk >= 0.0 else -1.0
        xk, yk = xk - d * yk * (2.0 ** -k), yk + d * xk * (2.0 ** -k)
        zk -= d * ATAN_TABLE[k]

    return sign * (yk / xk)

# --- Showcase ---
n = 25
xs = [-4.7, -3.0, -1.2, -0.8, -0.3, 0.0, 0.2, 0.7, 1.0, 1.2, 1.4, 1.55, 2.0, 3.0, 6.0, 10.0]
rows = []
for x in xs:
    approx = cordic_tan_n(x, n)
    truth = math.tan(x)  # gold standard for comparison only
    rows.append({"n": n, "x": x, "math.tan(x)": truth, "cordic_tan": approx, "abs_err": abs(approx - truth)})

df = pd.DataFrame(rows)
print(df.to_string(index=False))


 n     x  math.tan(x)    cordic_tan      abs_err
25 -4.70   -80.712763 -8.071259e+01 1.770096e-04
25 -3.00     0.142547  1.425465e-01 2.334807e-08
25 -1.20    -2.572152 -2.572152e+00 7.975933e-08
25 -0.80    -1.029639 -1.029639e+00 3.978765e-09
25 -0.30    -0.309336 -3.093363e-01 2.158001e-08
25  0.00     0.000000 -1.151196e-08 1.151196e-08
25  0.20     0.202710  2.027100e-01 1.277241e-08
25  0.70     0.842288  8.422883e-01 9.839059e-08
25  1.00     1.557408  1.557408e+00 3.335276e-08
25  1.20     2.572152  2.572152e+00 7.975933e-08
25  1.40     5.797884  5.797882e+00 1.305477e-06
25  1.55    48.078482  4.807847e+01 1.471186e-05
25  2.00    -2.185040 -2.185040e+00 3.702959e-08
25  3.00    -0.142547 -1.425465e-01 2.334807e-08
25  6.00    -0.291006 -2.910062e-01 3.927898e-08
25 10.00     0.648361  6.483608e-01 1.004715e-08


- **CORDIC idea (rotation mode).** Use micro-rotations by angles $\alpha_k=\arctan(2^{-k})$ with signs $d_k\in\{\pm1\}$ to drive the residual angle $z$ toward $0$:
  $$
  \begin{aligned}
  x_{k+1} &= x_k - d_k\,y_k\,2^{-k},\\
  y_{k+1} &= y_k + d_k\,x_k\,2^{-k},\\
  z_{k+1} &= z_k - d_k\,\arctan(2^{-k}),
  \end{aligned}
  \qquad d_k=\operatorname{sign}(z_k).
  $$
  Starting from $(x_0,y_0)=(1,0)$ and $z_0=\theta$, after $n$ steps $(x_n,y_n)\approx K_n(\cos\theta,\sin\theta)$.  
  Since the scale $K_n$ cancels in the ratio, $\tan\theta \approx y_n/x_n$ (no multiplications except by powers of $2$).

- **Argument reduction.** Reduce any $x$ to $t\in[0,\pi)$ via $x\bmod \pi$, then reflect to $[0,\tfrac{\pi}{2}]$ using $\tan(\pi - t)=-\tan(t)$ and keep the sign.

- **Computing $\pi$.** Because $\tan(\pi/4)=1$, find $a\in(0,\tfrac{\pi}{2})$ solving $\tan(a)=1$ by bisection with the CORDIC tangent; then $\pi\approx 4a$.
```


## Q2

In [55]:
import math
import pandas as pd

# --- CORDIC sine and cosine ---

# CORDIC scale factor invK(n) = 1 / Π sqrt(1 + 2^{-2k}); multiply (x,y) by invK at the end
def cordic_invK(n):
    invK = 1.0
    for k in range(n):
        invK *= 1.0 / math.sqrt(1.0 + 2.0**(-2*k))
    return invK


TWO_PI = 2.0 * math.pi

def cordic_sin_cos_n(theta, n):
    # Reduce to [-π, π]
    t = math.fmod(theta, TWO_PI)
    if t <= -math.pi:
        t += TWO_PI
    if t > math.pi:
        t -= TWO_PI

    # Map to [-π/2, π/2] using θ' = θ ± π and a global sign flip (sin,cos both change sign)
    sgn = 1.0
    if t > HALF_PI:
        t -= math.pi
        sgn = -1.0
    elif t < -HALF_PI:
        t += math.pi
        sgn = -1.0

    # CORDIC rotation mode
    xk, yk, zk = 1.0, 0.0, t
    for k in range(n):
        d = 1.0 if zk >= 0.0 else -1.0
        xk, yk = xk - d * yk * (2.0 ** -k), yk + d * xk * (2.0 ** -k)
        zk -= d * ATAN_TABLE[k]

    invK = cordic_invK(n)
    cos_val = sgn * (xk * invK)
    sin_val = sgn * (yk * invK)
    return sin_val, cos_val

def mysin_n(theta, n):
    s, c = cordic_sin_cos_n(theta, n)
    return s

def mycos_n(theta, n):
    s, c = cordic_sin_cos_n(theta, n)
    return c

# --- Showcase: compare to math.sin / math.cos on 0..1.6 (≈ up to π/2) ---

n = 25
xs = [k/10 for k in range(0, 17)]  # 0, 0.1, ..., 1.6

rows_sin, rows_cos = [], []
for x in xs:
    s_approx = mysin_n(x, n)
    c_approx = mycos_n(x, n)
    s_truth = math.sin(x)
    c_truth = math.cos(x)
    rows_sin.append({"n": n, "x": x, "math.sin(x)": s_truth, "cordic_sin": s_approx, "abs_err": abs(s_approx - s_truth)})
    rows_cos.append({"n": n, "x": x, "math.cos(x)": c_truth, "cordic_cos": c_approx, "abs_err": abs(c_approx - c_truth)})

df_sin = pd.DataFrame(rows_sin)
df_cos = pd.DataFrame(rows_cos)
print(df_sin.to_string(index=False))
print(df_cos.to_string(index=False))


 n   x  math.sin(x)    cordic_sin      abs_err
25 0.0     0.000000 -1.151196e-08 1.151196e-08
25 0.1     0.099833  9.983340e-02 1.816982e-08
25 0.2     0.198669  1.986693e-01 1.202374e-08
25 0.3     0.295520  2.955202e-01 1.881572e-08
25 0.4     0.389418  3.894183e-01 5.334560e-08
25 0.5     0.479426  4.794255e-01 5.115690e-08
25 0.6     0.564642  5.646425e-01 4.505129e-08
25 0.7     0.644218  6.442176e-01 4.402193e-08
25 0.8     0.717356  7.173561e-01 1.345545e-09
25 0.9     0.783327  7.833269e-01 5.636392e-09
25 1.0     0.841471  8.414710e-01 5.260685e-09
25 1.1     0.891207  8.912074e-01 1.544844e-09
25 1.2     0.932039  9.320391e-01 3.794846e-09
25 1.3     0.963558  9.635582e-01 1.094108e-08
25 1.4     0.985450  9.854497e-01 6.410094e-09
25 1.5     0.997495  9.974950e-01 3.115585e-09
25 1.6     0.999574  9.995736e-01 7.428207e-10
 n   x  math.cos(x)  cordic_cos      abs_err
25 0.0     1.000000    1.000000 4.440892e-16
25 0.1     0.995004    0.995004 1.823062e-09
25 0.2     0.980067

## Q3

In [56]:
# ==== CORDIC arctangent (vectoring mode), table built from Part-1 arctan ====
import math
import pandas as pd

# Part-1 arctan (argument halving) used ONLY to build the lookup table
def myatan_halving(x, n):
    t = float(x)
    for _ in range(n):
        t = t / (1.0 + math.sqrt(1.0 + t*t))
    t2 = t*t
    s = t - t*t2/3.0 + t*t2*t2/5.0 - t*t2*t2*t2/7.0 + t*t2*t2*t2*t2/9.0
    return s * (2 ** n)

# Build atan(2^-k) table from arctan
N_ATAN = 35
TABLE_LEN = 40
ATAN_TABLE = [myatan_halving(2.0**-k, N_ATAN) for k in range(TABLE_LEN)]

def cordic_atan(z, n):
    if z == 0.0:
        return 0.0
    sgn = 1.0
    if z < 0.0:
        sgn = -1.0
        z = -z
    recip = False
    if z > 1.0:                    # improve conditioning for large arguments
        recip = True
        z = 1.0 / z

    xk, yk, zacc = 1.0, z, 0.0
    for k in range(n):
        d = 1.0 if yk >= 0.0 else -1.0
        xk, yk = xk + d*yk*(2.0**-k), yk - d*xk*(2.0**-k)
        zacc += d * ATAN_TABLE[k]

    ang = zacc
    if recip:
        ang = (math.pi/2) - ang
    return sgn * ang

# --- Showcase ---
n = 25
xs = [-10, -3, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 3, 10]
rows = []
for x in xs:
    approx = cordic_atan(x, n)
    truth = math.atan(x)  # for comparison only
    rows.append({"n": n, "x": x, "math.atan(x)": truth, "cordic_atan": approx, "abs_err": abs(approx - truth)})

df = pd.DataFrame(rows)
print(df.to_string(index=False))




 n     x  math.atan(x)  cordic_atan      abs_err
25 -10.0     -1.471128    -1.471128 4.663336e-08
25  -3.0     -1.249046    -1.249046 3.899198e-08
25  -1.5     -0.982794    -0.982794 5.533680e-08
25  -1.0     -0.785398    -0.785398 2.830882e-08
25  -0.5     -0.463648    -0.463648 1.143782e-08
25   0.0      0.000000     0.000000 0.000000e+00
25   0.5      0.463648     0.463648 1.143782e-08
25   1.0      0.785398     0.785398 2.830882e-08
25   1.5      0.982794     0.982794 5.533680e-08
25   3.0      1.249046     1.249046 3.899198e-08
25  10.0      1.471128     1.471128 4.663336e-08


- **CORDIC arctan in Vectoring Mode**

 Given $ z $, compute $ \tan^{-1}(z) $ by driving $ y \to 0 $ starting from the initial vector:

$$
(x_0, y_0) = (1, z)
$$



$$
\begin{aligned}
x_{k+1} &= x_k + d_k \, y_k \, 2^{-k} \\
y_{k+1} &= y_k - d_k \, x_k \, 2^{-k} \\
z_{\text{acc}, k+1} &= z_{\text{acc}, k} + d_k \, \tan^{-1}(2^{-k})
\end{aligned}
$$

where

$$
d_k = \operatorname{sign}(y_k)
$$


## Q4

In [57]:
# ==== arcsine via CORDIC, based on CORDIC arctangent ====
import math
import pandas as pd

# Reuse the same table builder and CORDIC atan (include here for standalone cell)
def myatan_halving(x, n):
    t = float(x)
    for _ in range(n):
        t = t / (1.0 + math.sqrt(1.0 + t*t))
    t2 = t*t
    s = t - t*t2/3.0 + t*t2*t2/5.0 - t*t2*t2*t2/7.0 + t*t2*t2*t2*t2/9.0
    return s * (2 ** n)

N_ATAN = 30
TABLE_LEN = 40
ATAN_TABLE = [myatan_halving(2.0**-k, N_ATAN) for k in range(TABLE_LEN)]

def cordic_atan(z, n):
    if z == 0.0:
        return 0.0
    sgn = 1.0
    if z < 0.0:
        sgn = -1.0
        z = -z
    recip = False
    if z > 1.0:
        recip = True
        z = 1.0 / z
    xk, yk, zacc = 1.0, z, 0.0
    for k in range(n):
        d = 1.0 if yk >= 0.0 else -1.0
        xk, yk = xk + d*yk*(2.0**-k), yk - d*xk*(2.0**-k)
        zacc += d * ATAN_TABLE[k]
    ang = zacc
    if recip:
        ang = (math.pi/2) - ang
    return sgn * ang

def cordic_asin(x, n):
    if not (-1.0 <= x <= 1.0):
        return float('nan')
    if x == 1.0:
        return math.pi/2
    if x == -1.0:
        return -math.pi/2
    if x == 0.0:
        return 0.0
    denom = math.sqrt(max(0.0, 1.0 - x*x))
    if denom == 0.0:
        return math.copysign(math.pi/2, x)
    t = x / denom
    return cordic_atan(t, n)

# --- Showcase ---
n = 25
xs = [-1.0, -0.95, -0.5, -0.1, 0.0, 0.1, 0.5, 0.95, 1.0]
rows = []
for x in xs:
    approx = cordic_asin(x, n)
    truth = math.asin(x)  # for comparison only
    rows.append({"n": n, "x": x, "math.asin(x)": truth, "cordic_asin": approx, "abs_err": abs(approx - truth)})

df = pd.DataFrame(rows)
print(df.to_string(index=False))


 n     x  math.asin(x)  cordic_asin      abs_err
25 -1.00     -1.570796    -1.570796 0.000000e+00
25 -0.95     -1.253236    -1.253236 7.653094e-09
25 -0.50     -0.523599    -0.523599 3.218391e-08
25 -0.10     -0.100167    -0.100167 4.962074e-08
25  0.00      0.000000     0.000000 0.000000e+00
25  0.10      0.100167     0.100167 4.962074e-08
25  0.50      0.523599     0.523599 3.218391e-08
25  0.95      1.253236     1.253236 7.653094e-09
25  1.00      1.570796     1.570796 0.000000e+00


- **arcsin based on arctan identity (using CORDIC arctan)**

The arcsine can be expressed in terms of the arctangent as:

$ \arcsin(x) = \arctan\!\left( \dfrac{x}{\sqrt{1 - x^2}} \right) $

where the principal value lies in the interval $[-\pi/2, \pi/2]$.
